Install neccessary packages 

In [1]:
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [1]:
from urllib import request as rq
import pandas as pd
import os
import pyarrow as pa # not sure will i use this 
import numpy as np
import ipywidgets
from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed, VBox
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder

Download the Dublin Bus data from https://data.gov.ie/organization/dublin-city-council?tags=Transport+and+Infrastructure

Use a generator to load the data into pandas data frame https://pandasninja.com/2019/04/how-to-read-lots-of-csv-files-easily-into-pandas/#:~:text=How%20to%20read%20lots%20of%20csv%20files%20easily,need%20...%204%204.%20Leverage%20regular%20expressions%20


In [17]:
# Function to load the Dublin bus gz files 
def load_Files(files):
    columns = ['Timestamp', 'LineID', 'Direction', 'JourneyPatternID', 'TimeFrame', 'VehicleJourneyID', 'Operator', 'Congestion', 'LonWGS84', 'LatWGS84', 'Delay', 'BlockID', 'VehicleID', 'StopID', 'AtStop']
    for f in files:
        print(f)
        yield pd.read_csv('./Data/Bus/Gz/' + f, compression='gzip', delimiter=',', header=0, names=columns, parse_dates=True, low_memory=True)

# Check parquet file existence before downloading - iof starting from afresh this takes a long time
if os.path.exists('./Data/Bus/CleanedBusData.parquet'):
    print("Parquest file exists, means the Bus data has been downloaded already ")
    df = pd.read_parquet('./Data/Bus/CleanedBusData.parquet')
else:
    os.makedirs('./Zips/Bus/', exist_ok = True)
    url = "https://opendata.dublincity.ie/TrafficOpenData/sir010113-310113.zip"
    busFile = rq.urlretrieve(url, './Zips/Bus/DublinBusdata.zip' )  
    import zipfile as zip
    # Zip creates its own folders - no need to check for folder existence
    with zip.ZipFile(busFile[0],  mode='r') as arc: 
        arc.extractall('./Data/Bus/Gz/')  
    files = os.listdir('./Data/Bus/Gz/')
    DBfiles = [f for f in files if f.endswith('.gz')]
    df = pd.concat(load_Files(DBfiles))



siri.20130101.csv.gz
siri.20130102.csv.gz
siri.20130103.csv.gz
siri.20130104.csv.gz
siri.20130105.csv.gz
siri.20130106.csv.gz
siri.20130107.csv.gz
siri.20130108.csv.gz
siri.20130109.csv.gz
siri.20130110.csv.gz
siri.20130111.csv.gz
siri.20130112.csv.gz
siri.20130113.csv.gz
siri.20130114.csv.gz
siri.20130115.csv.gz
siri.20130116.csv.gz
siri.20130117.csv.gz
siri.20130118.csv.gz
siri.20130119.csv.gz
siri.20130120.csv.gz
siri.20130121.csv.gz
siri.20130122.csv.gz
siri.20130123.csv.gz
siri.20130124.csv.gz
siri.20130125.csv.gz
siri.20130126.csv.gz
siri.20130127.csv.gz
siri.20130128.csv.gz
siri.20130129.csv.gz
siri.20130130.csv.gz
siri.20130131.csv.gz


In [33]:
ColumnsToExclude = ('JourneyPatternID', 'StopID')

# Data is loaded - now needs to be cleaned, there is an argument to be made to include this in the above cell 
# but I have added it here so that I can add more columns to be excluded as I go rather than load the raw data each time
for c in ColumnsToExclude:
    if c in df.columns:
        df = df.drop(c, axis=1)
    else:
        print('Columns already dropped')



In [47]:
# Getting the list of column Types
CategoricalColumns = list(set(list(df.select_dtypes(exclude=[np.number]).columns)))
NumericalColumns = list(set(list(df.select_dtypes([np.number]).columns)))
#objectTypeColumns = list(set(list(df.select_dtypes([np.object_]).columns)))
DateColumns = list(set(list(df.select_dtypes([np.datetime64]).columns)))


#print(objectTypeColumns)


CategoricalColumns = list(set(CategoricalColumns) - set(ColumnsToExclude)-set(DateColumns))
NumericalColumns = list(set(NumericalColumns)-set(ColumnsToExclude)-set(DateColumns))

print(CategoricalColumns)
print(NumericalColumns)
print(DateColumns)


['TimeFrame', 'Operator', 'Time']
['Congestion', 'LonWGS84', 'AtStop', 'VehicleJourneyID', 'LatWGS84', 'LineID', 'Timestamp', 'BlockID', 'Delay', 'Direction', 'VehicleID']
['datetime']


In [48]:
df.head()

,Timestamp,LineID,Direction,TimeFrame,VehicleJourneyID,Operator,Congestion,LonWGS84,LatWGS84,Delay,BlockID,VehicleID,AtStop,datetime,Time
0,1356998405000000,27.0,0,2012-12-31,3883,RD,0,-6.233417,53.342232,0,27017,33521,0,2013-01-01 00:00:05,00:00:05
1,1356998407000000,40.0,0,2012-12-31,2226,HN,0,-6.278250,53.416683,0,40206,33142,0,2013-01-01 00:00:07,00:00:07
2,1356998407000000,7.0,0,2012-12-31,6106,D1,0,-6.231633,53.317768,0,7019,43004,1,2013-01-01 00:00:07,00:00:07
3,1356998411000000,747.0,0,2012-12-31,3531,SL,0,-6.254617,53.355484,-454,747007,40039,0,2013-01-01 00:00:11,00:00:11
4,1356998411000000,56.0,0,2012-12-31,1830,RD,0,-6.233183,53.342201,0,56001,33488,0,2013-01-01 00:00:11,00:00:11


In [49]:
# Dealt with the timestamp and make it human readable
df['datetime'] = pd.to_datetime(df['Timestamp'], unit='us')
df['Time'] = df['datetime'].dt.time

In [26]:
print(df.head)
print(df.Time)

<bound method NDFrame.head of                Timestamp  LineID  Direction JourneyPatternID   TimeFrame  \
0       1356998405000000    27.0          0              NaN  2012-12-31   
1       1356998407000000    40.0          0              NaN  2012-12-31   
2       1356998407000000     7.0          0         00071003  2012-12-31   
3       1356998411000000   747.0          0         07471001  2012-12-31   
4       1356998411000000    56.0          0         056A1001  2012-12-31   
...                  ...     ...        ...              ...         ...   
603915  1359632694000000    39.0          0         039A1001  2013-01-31   
603916  1359632694000000     9.0          0         00090001  2013-01-31   
603917  1359632694000000    41.0          0         00410001  2013-01-31   
603918  1359632694000000   145.0          0         084X1001  2013-01-31   
603919  1359632694000000   145.0          0         01450001  2013-01-31   

        VehicleJourneyID Operator  Congestion  LonWGS84  

In [50]:
print ('The data has {} Rows and {} columns'.format(df.shape[0],df.shape[1]))
print("The types of columns are:")
df.dtypes

The data has 44455102 Rows and 15 columns
The types of columns are:


Timestamp                    int64
LineID                     float64
Direction                    int64
TimeFrame                   object
VehicleJourneyID             int64
Operator                    object
Congestion                   int64
LonWGS84                   float64
LatWGS84                   float64
Delay                        int64
BlockID                      int64
VehicleID                    int64
AtStop                       int64
datetime            datetime64[ns]
Time                        object
dtype: object

In [51]:
def num_missing(x):
    return len(x.index)-x.count()

def num_unique(x):
    return len(np.unique(x))

temp_df = df.describe().T
missing_df = pd.DataFrame(df.apply(num_missing, axis=0)) 
missing_df.columns = ['missing']
unq_df = pd.DataFrame(df.apply(num_unique, axis=0))
unq_df.columns = ['unique']
types_df = pd.DataFrame(df.dtypes)
types_df.columns = ['DataType']

In [54]:
df['Time'].unique()

array([datetime.time(0, 0, 5), datetime.time(0, 0, 7),
       datetime.time(0, 0, 11), ..., datetime.time(2, 52, 46),
       datetime.time(6, 4, 39), datetime.time(6, 26, 18)], dtype=object)

In [55]:
summary_df = temp_df.join(missing_df).join(unq_df).join(types_df)
summary_df

,count,mean,std,min,25%,50%,75%,max,missing,unique,DataType
Timestamp,44455102.0,1.358318e+15,7.394928e+11,1.356998e+15,1.357679e+15,1.358327e+15,1.358955e+15,1.359633e+15,0,813103,int64
LineID,44453044.0,7.796879e+01,1.146419e+02,1.000000e+00,2.500000e+01,4.000000e+01,8.300000e+01,7.470000e+02,2058,67,float64
Direction,44455102.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,1,int64
VehicleJourneyID,44455102.0,9.417122e+03,6.159844e+04,1.000000e+00,2.536000e+03,4.718000e+03,6.769000e+03,9.998560e+05,0,18614,int64
Congestion,44455102.0,1.163261e-02,1.072254e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0,2,int64
LonWGS84,44455102.0,-6.272803e+00,8.389025e-02,-6.617517e+00,-6.308650e+00,-6.261600e+00,-6.233166e+00,-6.052917e+00,0,456755,float64
LatWGS84,44455102.0,5.334513e+01,5.488354e-02,5.306802e+01,5.332005e+01,5.334645e+01,5.337533e+01,5.360873e+01,0,131327,float64
Delay,44455102.0,-2.816513e+01,4.729678e+02,-1.504500e+04,-2.090000e+02,0.000000e+00,1.040000e+02,1.161220e+05,0,12638,int64
BlockID,44455102.0,1.092438e+05,1.921244e+05,3.900000e+02,1.602000e+04,4.020500e+04,8.400400e+04,8.350020e+05,0,1238,int64
VehicleID,44455102.0,3.542925e+04,3.281093e+03,2.804700e+04,3.330800e+04,3.352500e+04,3.802500e+04,4.307800e+04,0,911,int64


In [56]:
col_names = list(types_df.index)
num_cols = len(col_names)
index = range(num_cols)
cat_index = []
for i in index:
    if col_names[i] in CategoricalColumns:
        cat_index.append(i)
summary_df_cat = missing_df.join(unq_df).join(types_df.iloc[cat_index], how='inner') #Only summarize categorical columns
summary_df_cat

,missing,unique,DataType
TimeFrame,0,32,object
Operator,0,8,object
Time,0,72014,object


In [57]:
df.to_parquet('./Data/Bus/CleanedBusData.parquet')

Download the weather station details https://cli.fusio.net/cli/climate_data/webdata/StationDetails.csv

In [ ]:
url = 'https://cli.fusio.net/cli/climate_data/webdata/StationDetails.csv'


Load the weather statio details for Dublin

In [ ]:
weatherStations = pd.read_csv(url)
weatherStations = weatherStations['county'].str.contains('Dublin')
weatherStations.head(10)

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
Name: county, dtype: bool